In [1]:
# essential libraries
import os
import math
import numpy as np
import pandas as pd
import datetime as dt

# prediction model
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression

# hide warnings
import warnings
warnings.filterwarnings('ignore')

# visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from PIL import Image

# API, URL, DB
from bs4 import BeautifulSoup
import requests
from urllib.parse import urlencode
from urllib.parse import quote_plus
from urllib.request import urlopen
import sqlite3




# converter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()   

#pandas와 같이 plotly에서도 간단한 시각화 지원
import cufflinks as cf
cf.go_offline(connected=True)

# for offline ploting
import plotly
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly.io as pio
pio.renderers.default = "notebook_connected"

In [2]:
print(px.colors.qualitative.Prism)

['rgb(95, 70, 144)', 'rgb(29, 105, 150)', 'rgb(56, 166, 165)', 'rgb(15, 133, 84)', 'rgb(115, 175, 72)', 'rgb(237, 173, 8)', 'rgb(225, 124, 5)', 'rgb(204, 80, 62)', 'rgb(148, 52, 110)', 'rgb(111, 64, 112)', 'rgb(102, 102, 102)']


In [3]:
# color pallette
cnf, dth, rec, act = '#393e46', '#ff2e63', '#21bf73', '#fe9801' 
DEFAULT_PLOTLY_COLORS=['#636EFA', '#EF553B', 
                       '#00CC96', '#AB63FA',
                       '#FFA15A', '#19D3F3',
                       '#FF6692', '#B6E880',
                       '#FF97FF', '#FECB52','rgb(57,105,172)']

DEFAULT_PLOTLY_COLORS2 = ['rgb(230,210,0)', 'rgb(241,105,19)','rgb(8,104,172)', 'rgb(0,109,44)', 'rgb(131,75,160)']
DEFAULT_PLOTLY_COLORS2_2 = ['rgb(230,210,0)', 'rgb(241,105,19)', 'rgb(241,105,19)', 'rgb(8,104,172)', 'rgb(8,104,172)', 'rgb(0,109,44)','rgb(0,109,44)', 'rgb(131,75,160)', 'rgb(131,75,160)', 'rgb(131,75,160)']

layout_setting = {'xaxis_title':'Date',
                  'yaxis_title':'Number',
                  'font':dict(size=18,color='#60606e',
                              family='Franklin Gothic')}

layout_font = {'font':dict(size=18,color='#60606e',
                           family='Franklin Gothic' )}

In [4]:
path = '../covid'
os.chdir('./data')
file_name_list = os.listdir()

In [5]:
file_name_list

['202003Korea_Population.csv',
 '20200428ncovProvince.csv',
 '20200428_Gender.csv',
 'avgAgeProvince.jpg',
 'Case.csv',
 'COVID-19 Cases.csv',
 'floating_data_2019_03.db',
 'floating_data_2020.db',
 'floating_data_Total.db',
 'NaverSearchTrend.csv',
 'PatientInfo.csv',
 'PatientRoute.csv',
 'PatientTrend_20200414_20200427.xlsx',
 'Policy.csv',
 'Region.csv',
 'SearchTrend.csv',
 'SeoulFloating.csv',
 'Time.csv',
 'TimeAge.csv',
 'TimeConfirmed_SCJ.csv',
 'TimeGender.csv',
 'TimeProvince.csv',
 'Weather.csv',
 'WHO-COVID-19-global-data.csv',
 '노선별 수송현황 0221-0227.xlsx',
 '노선별 수송현황 0228-0305.xlsx',
 '노선별 수송현황 0306-0312.xlsx',
 '노선별 수송현황 0313-0319.xlsx',
 '노선별 수송현황 0320-0326.xlsx',
 '노선별 수송현황 0327-0402.xlsx',
 '노선별 수송현황 0403-0409.xlsx',
 '노선별 수송현황 0410-0416.xlsx',
 '노선별 수송현황 0417-0423.xlsx',
 '노선별 수송현황 0424-0430.xlsx',
 '노선별 수송현황 0501-0507.xlsx',
 '주단위 교통량.xlsx']

In [6]:
case = pd.read_csv("Case.csv")
region = pd.read_csv("Region.csv")
patientinfo = pd.read_csv("PatientInfo.csv")
route = pd.read_csv( 'PatientRoute.csv')
timeProve = pd.read_csv("TimeProvince.csv")
timeAge = pd.read_csv("TimeAge.csv")
timeGender = pd.read_csv("TimeGender.csv")
seoulFloating = pd.read_csv('SeoulFloating.csv')
population = pd.read_csv("202003Korea_Population.csv", encoding='ANSI') # 2020.03 한국 인구수 from 통계청
numProvince = pd.read_csv("20200428ncovProvince.csv", index_col=False) # 2020.04.28 질병관리본부데이터
# http://ncov.mohw.go.kr/tcmBoardView.do?brdId=&brdGubun=&dataGubun=&ncvContSeq=354256&contSeq=354256&board_id=140&gubun=BDJ

---

# Topic 2. 과연, COVID19는 누구에게 치명적일까?

> 대부분의 사람들은 고연령층과 기저질환이 있는 사람들이 높은 사망률을 가지고 있다는 점은 모두 알고 있습니다. 하지만 정확히 어떤 연령대가 제일 취약하고 실제 기저질환을 가진 사람들이 어떠한 경로로 감염이 되는지, 사망률이 면역력과 연관이 있는지에 대한 부분은 알지 못합니다. 그래서 나이와 질병유무를 중심으로 환자 위험 요인에 대한 분석을 진행하였습니다. 이를 위해 연령별 완치기간과 치명률을 통해 알아보고, 세부적으로 사망원인을 파악하며 코로나 바이러스에 가장 취약한 집단을 도출하고자 이번 주제의 분석 작업을 진행하였습니다.

## 결론 및 요약
 
 COVID19 바이러스는 고연령일수록 완치기간과 사망률이 높습니다, 또한 거의 모든 사망자는 기저질환을 가지고 있었으며 그 중 호흡기계 질환(폐 질환)보다 다른 순환기계나 내분비계 기저질환을 앓고 있는 환자에게 더 높은 사망률을 보입니다.

### 1. 나이가 많을 수록 완치 기간이 오래 걸린다? 

COVID19를 회복하기 까지의 기간은 평균 21일, 약 3주로 회복하기까지의 기간이 깁니다. 특히 <span style="color:red">고연령층은</span> 다른 연령대에 비해 완치기간이 길었으며 이는 나이에 따른 면역력 차이라고 볼 수 있었습니다.
 
 반대로 20대는 빠른 회복기간을 가진다는 것은 면역력이 높다는 것으로 예상할 수 있습니다. 또한, 20대의 경우 기존의 병력이 있는 경우가 드물고, 코로나가 합병증과 함께 사망으로까지 이어질 확률이 낮을 것입니다. 
 
### 2. 치명률과 사망원인을 알아보자.

첫번째 치명률 분석을 통해 알 수 있었던 점은 4월 30일 기준으로 80대 이상 확진자의 치명률은 약 25%로 가장 높았습니다. 또한 치명률은 연령에 비례함을 알 수 있었습니다.

두번째, 사망자를 파악해보니 거의 모두 기저질환을 가지고 있는 고연령층이었습니다. 총 247명의 사망자중 244명(98.8%)은 기저질환이 있는 환자였으며 흔히 알고 있는 페렴 환자에만 높은 사망률을 보일 뿐만 아니라 소화기계 질환, 순환기계 질환 등 다른 기저 질환을 앓고 있는 환자들의 사망률이 더 높았음을 알 수 있었습니다. 또한 사망자들의 53%는 병원 및 요양원에서 감염 되었음을 보아 고연령층이 자주 방문하는 병원내에서의 감염이 잘 된다는 것을 알 수 있었습니다.

### 3. 해결방안

노년층의 사망률을 줄이기 위해서는 확진자가 있는 병원의 방문을 삼가해야 합니다. 이를 위해 노년층을 대상으로 현재 확진자가 분포한 지역의 의료기관 및 요양시설에 대한 정보를 알기 쉽게 안내해 주는 알림 서비스 개발도 필요하다고 생각합니다. 또한 병원 및 요양 시설에 대한 방역 작업이 우선적으로 철저하게 이루워져야 노년층에게 치명적인 코로나 바이러스를 예방할 수 있으며 이에 따른 사망률을 줄일 수 있습니다.

In [42]:
fig = make_subplots(1,2, horizontal_spacing=0.05, subplot_titles=['<b>연령별 누적 확진자수</b>', '<b>연령별 누적 사망자수</b>'])
trace1 = px.bar(timeAge, x='date', y='confirmed',
                 hover_data=['age'], color='age', )
trace2 = px.bar(timeAge, x='date', y='deceased',
                 hover_data=['age'], color='age', )

for i in range(len(trace1['data'])):
    trace2['data'][i].showlegend=False
    fig.add_trace(trace1['data'][i], 1, 1)
    fig.add_trace(trace2['data'][i], 1, 2)
    

fig.update_yaxes(title_text="<b>Number</b>", row=1, col=1)
fig.update_xaxes(title_text="<b>Date</b>")

fig.update_layout(title='<b>연령별 누적 추이 비교</b>', 
                  barmode='stack', **layout_font, height=600, width=900)
fig.show()

우선, 연령별로 확진 환자와 사망 환자가 얼마나 분포 현황에 알아보기 위한 EDA를 진행했고 아래와 같은 연령별 확진자와 사망자 추이 결과를 알 수 있었습니다.

연령별 누적 확진자 추이
- 20대, 50대가 가장 높은 확진자 비율을 차지 하고 있었습니다.
- 20대, 50대, 40대, 60대, 30대, 70대, 10대, 80대, 0대 순으로 높은 확진자 비율을 가지고 있습니다. 
- 20대는 확진 환자 수가 많지만, 사망 환자는 없습니다.
- 반대로, 80대는 확진 환자 수는 적지만 사망 환자 수가 가장 많습니다.

연령별 누적 사망자 추이
- 나이때가 높은 순으로 사망자 수가 많습니다.
- 70대,80대의 사망자 추이는 꾸준히 증가하는것을 볼 수 있습니다.


 그 다음은 환자 정보를 통해 연령대 별로 격리 중인 환자, 격리 해제된 환자 그리고 사망 환자에 대한 비교를 위한 작업을 하였습니다. 
 
 #### 다음은 2020.01.23 - 2020.04.14 사이의 연령별 환자 상태 입니다.

In [8]:
patientinfo

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
0,1000000001,2.0,male,1964.0,50s,Korea,Seoul,Gangseo-gu,NaN,overseas inflow,1.0,NaN,75.0,2020-01-22,2020-01-23,2020-02-05,NaN,released
1,1000000002,5.0,male,1987.0,30s,Korea,Seoul,Jungnang-gu,NaN,overseas inflow,1.0,NaN,31.0,NaN,2020-01-30,2020-03-02,NaN,released
2,1000000003,6.0,male,1964.0,50s,Korea,Seoul,Jongno-gu,NaN,contact with patient,2.0,2.002000e+09,17.0,NaN,2020-01-30,2020-02-19,NaN,released
3,1000000004,7.0,male,1991.0,20s,Korea,Seoul,Mapo-gu,NaN,overseas inflow,1.0,NaN,9.0,2020-01-26,2020-01-30,2020-02-15,NaN,released
4,1000000005,9.0,female,1992.0,20s,Korea,Seoul,Seongbuk-gu,NaN,contact with patient,2.0,1.000000e+09,2.0,NaN,2020-01-31,2020-02-24,NaN,released
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,7000000009,9651.0,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,14.0,NaN,2020-03-29,NaN,NaN,isolated
3384,7000000010,NaN,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,18.0,NaN,2020-04-03,NaN,NaN,isolated
3385,7000000011,NaN,male,NaN,30s,Korea,Jeju-do,Jeju-do,NaN,contact with patient,NaN,7.000000e+09,5.0,NaN,2020-04-03,NaN,NaN,isolated
3386,7000000012,NaN,female,NaN,20s,Korea,Jeju-do,Jeju-do,NaN,overseas inflow,NaN,NaN,9.0,NaN,2020-04-03,NaN,NaN,isolated


In [9]:
state_age=patientinfo.groupby('state')['age'].value_counts()
stage_age=pd.DataFrame(state_age)

In [10]:
state_age=state_age.copy()

In [11]:
stage_age.rename(index={'0s':'0대','10s':'10대','20s':'20대','30s':'30대','40s':'40대','50s':'50대','60s':'60대','70s':'70대','80s':'80대','90s':'90대','100s':'100대'},inplace=True)

stage_age=stage_age.T
stage_age=stage_age.drop(('isolated','100대'), axis=1, inplace=False) #격리된 연령층 100살대 1개 제외
stage_age=stage_age.T

stage_age.columns=['count']
released_=stage_age.loc['released',:].sort_index()
isolated=stage_age.loc['isolated',:].sort_index()
deceased=stage_age.loc['deceased',:].sort_index()
deceased_t=pd.DataFrame(data=(0,0,0), index=['0대','10대','20대'])
deceased_t.columns=['count']
deceased=pd.concat([deceased_t, deceased], axis=0)

In [12]:
layout_setting = {'xaxis_title':'연령별','font':dict(size=18,color='#60606e',family='Franklin Gothic' )}

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(go.Bar(x=released_.index, y=released_['count'],
                    name='격리해제 환자'), secondary_y = False)
fig.add_trace(go.Bar(x=isolated.index, y=isolated['count'],
                    name='격리 환자'), secondary_y = False)
fig.add_trace(go.Scatter(x=deceased.index, y=deceased['count'], 
                    mode='lines+markers', name='사망 환자', line=dict(
                color="Green",
                width=2,
            )), secondary_y=True)

fig.update_layout(title='<b>연령별 환자 상태 (격리해제 환자/ 격리 환자 / 사망 환자)<b>', **layout_font, xaxis=dict(showgrid=False, zeroline=False),
    yaxis=dict(showgrid=False, zeroline=False))

fig.update_yaxes(title_text='<b>환자수</b>', secondary_y=False, tickfont=dict(size=8))
fig.update_yaxes(title_text='<b>사망자수</b>', secondary_y=True, tickfont=dict(size=8))

fig.show()

확진환자 수는 20대에 가장 두드러지게 볼 수 있었으며, 사망자수는 80대에 가장 많았음을 알 수 있었습니다.
가장 주목해서 볼 수 있었던 점은 70대부터 90대는 격리해제 환자보다 아직 격리중인 환자수가 더 많다는 사실을 볼 수 있었습니다.

반대로 0대부대 50대는 격리해제 환자가 격리 환자보다 많았습니다. 이는 70대부터는 완치가 잘 되지 않고 격리 해제가 되기까지의 기간이 길기 때문에 격리 환자 수가 격리 해제 환자보다 많다라고 예상해 볼 수 있었습니다.

## 1.  나이가 많을 수록 완치기간이 더 오래걸린다?

 코로나가 연령대 별로 얼마나 오랜 기간동안 완치되지 않은지에 대해 알아보고자 하였습니다. 더불어, 사망률이 코로나의 완치기간과 연관성이 있는가에 대한 의문점이 들었습니다. 완치기간은 '확진날짜(Confirmed date)'와 '격리 해제(Released date)'의 차이로 계산하였습니다.

 PatientInfo data에서 격리해제일과 확진판정일의 차이를 구해서 실제 나이가 많을 수록 면역력이 부족하기 때문에 격리 해제까지 기간이 오래 걸리는가에 대해 알아보았습니다. 우선 격리해제자인데 격리해제된일이 NaN인 사람, 즉 Null값을 가지고 있는 304명의 데이터는 제외하고 분석작업을 진행하였으며, 치유기간의 평균을 구해 연령대별 평균 이상, 이하의 분포를 알아보았습니다.

In [13]:
released_p=patientinfo[patientinfo['state']=='released'] #격리해제자인데 격리해제된날 NaN인 사람 : 304명 제외 
released_p=released_p.dropna(subset=['released_date','age'], inplace=False)

released_p['confirmed_date']=pd.to_datetime(released_p['confirmed_date'])
released_p['released_date']=pd.to_datetime(released_p['released_date'])
released_p['period_diff']=(released_p['released_date'])-(released_p['confirmed_date']) #격리해제일 - 확진판정일

In [14]:
released_p['period_diff'].describe()

count                       1312
mean     22 days 10:01:27.804878
std       9 days 19:40:21.937886
min           -15 days +00:00:00
25%             16 days 00:00:00
50%             22 days 00:00:00
75%             28 days 00:00:00
max             68 days 00:00:00
Name: period_diff, dtype: object

### 1-1) 평균 치유기간

평균 치유기간은 21일, 최대 54일, 최소 0일이라는 결과가 나왔습니다. 그래서 21일을 기준으로 이상, 이하으로 분류하여 각 연령층의 비율을 알아보았습니다. 

아래의 그래프에서도 볼 수 있듯이 0대와 10대가 전반적으로 훨씬 빠르게 완치되는 것을 알 수 있습니다. 반대로 80, 90대 같은 경우 평균 완치기간을 넘는 환자 비율이 63.3%, 70%를 차지하였습니다. 또한 모든 연령은 평균 21일동안 코로나로 인한 질병을 앓았고, 코로나는 감기처럼 쉽게 낫는 질병이 아님을 알 수 있었습니다.

In [15]:
released_p['over_aveg']=np.where(released_p['period_diff']>'21 days',1,0)
over_av_released=released_p[released_p['over_aveg']==1]
under_av_released=released_p[released_p['over_aveg']==0]

over_av=pd.DataFrame(over_av_released['age'].value_counts().sort_index()).reset_index()
under_av=pd.DataFrame(under_av_released['age'].value_counts().sort_index()).reset_index()

#연령대층별로 감염자수가 확연히 다르기때문에 각 연령층별의 비율로 계산
under_av['per']=under_av['age']/(under_av['age']+over_av['age']) 
over_av['per']=over_av['age']/(under_av['age']+over_av['age'])

#컬럼 재정리
under_av.columns=['age', 'count', 'under_per']
over_av.columns=['age', 'count', 'over_per']

In [16]:
age_avg_released=pd.merge(under_av,over_av, on='age')

In [17]:
fig = make_subplots(rows=1, cols=1, horizontal_spacing=0.03, vertical_spacing= 0.05)
age_avg_released = age_avg_released.loc[list(reversed(age_avg_released.index))]

fig.add_trace(go.Bar(x=age_avg_released.under_per, y=age_avg_released.age, name='빠른 치유 기간',
                     orientation='h'))

fig.add_trace(go.Bar(x=age_avg_released.over_per, y=age_avg_released.age, name='오랜 치유 기간',
                     text=age_avg_released.over_per, texttemplate='%{x:.1%}', textposition='inside',
                    textfont=dict(color='white'),
                    orientation='h'))
fig.update_layout(barmode='stack',
                  paper_bgcolor='rgb(248, 248, 255)',
                  plot_bgcolor='rgb(248, 248, 255)',
                 )

fig.update_layout(
    title_text="<b>연령별 평균완치기간 이상/이하 비율 <b>", **layout_font)
    
fig.update_xaxes(title_text="<b>비율</b>")
fig.update_yaxes(title_text="<b>연령</b>")
fig.show()

### 1-2) 연령대별 평균 치유 기간

고연령층의 확실한 치유기간 비교를 위해 총 5개의 연령대와 연령층별로 나누어 비교해보았습니다. 

연령대 지표(한국보건사회 논문 참고)는 유아층은 0-9세, 청년층은 10-29세, 중년층은 30-49세, 장년층은 50-69세, 그리고 노년층은 70세 이상으로 분류하였습니다. 분석 결과 각 연령대 별로 치유기간의 차이가 있었습니다.가장 눈에 띄는 것은 노년층이 확연히 다른 연령대층에 비해 치유기간이 길다라는 점을 볼 수 있스며 고연령층으로 갈수록 치유기간이 길어지는 것 또한 볼 수 있었습니다.

In [18]:
conditions  = [ released_p['age'] == '0s',  
               (released_p['age'] == '10s') | (released_p['age'] == '20s'), 
               (released_p['age'] == '30s') | (released_p['age'] == '40s'),
               (released_p['age'] == '50s') | (released_p['age'] == '60s'),
               (released_p['age'] == '70s')  | (released_p['age'] == '80s') | (released_p['age'] == '90s')]
choices     = [ '유아층', '청년층', '중년층', '장년층','노년층']

In [19]:
released_p["age_class"] = np.select(conditions, choices, default=np.nan)

In [20]:
released_p['period_diff']=released_p['period_diff'].astype(str)

In [21]:
import re

diff_date_list=[]

for diff in released_p['period_diff']:
    pattern=re.compile('[\d*]{1,2}')
    word=re.search(pattern=pattern,string=diff)
    diff_date_list.append(word.group())
    
released_p['period_diff']=diff_date_list

In [22]:
released_p['period_diff']
released_p['period_diff']=released_p['period_diff'].astype(int)

In [23]:
age_class_mean=pd.DataFrame(released_p.groupby('age_class')['period_diff'].mean(), index=['유아층','청년층','중년층','장년층','노년층']).reset_index()

In [24]:
age_class_mean

,index,period_diff
0,유아층,20.357143
1,청년층,21.660050
2,중년층,22.010309
3,장년층,22.825000
4,노년층,25.775701


In [25]:
age_mean=pd.DataFrame(released_p.groupby('age')['period_diff'].mean()).reset_index()

In [26]:
age_mean

,age,period_diff
0,0s,20.357143
1,10s,20.418182
2,20s,21.856322
3,30s,21.915254
4,40s,22.090047
5,50s,22.076336
6,60s,24.246377
7,70s,24.055556
8,80s,27.595238
9,90s,27.272727


In [27]:
age_class_mean.columns=['연령대','평균 완치기간']
age_mean.columns=['연령','평균 완치기간']

In [28]:
age_class_mean['평균 완치기간']

0    20.357143
1    21.660050
2    22.010309
3    22.825000
4    25.775701
Name: 평균 완치기간, dtype: float64

In [29]:
layout_setting = {'xaxis_title':'<b>연령대별</b>','yaxis_title':'<b>평균 완치기간</b>',
                  'font':dict(size=18,color='#60606e',family='Franklin Gothic' )}

fig = make_subplots(rows=1, cols=2, horizontal_spacing=0.15, vertical_spacing=0.05)

fig.add_trace(go.Bar(x=age_class_mean['연령대'],y=age_class_mean['평균 완치기간'],
                     marker=dict(color=DEFAULT_PLOTLY_COLORS2)), row=1,col=1)

fig.add_trace(go.Bar(x=age_mean['연령'],y=age_mean['평균 완치기간'], 
                     marker=dict(color=DEFAULT_PLOTLY_COLORS2_2)
                    ),row=1, col=2)

fig.update_layout(title='<b>연령대별 / 연령별 평균 완치기간<b>', **layout_setting, showlegend=False)
fig.update_yaxes( range=[15,25])

fig.update_xaxes(title_text='<b>연령별</b>', row=1, col=2)
fig.update_yaxes(title_text='<b>평균 완치기간</b>', row=1, col=2, range=[15,25])

## 2. 치명률과 사망 환자 원인 분석

### 2-1) 치명률 

앞에서 고연령층의 높은 사망자수와 긴 완치기간을 확인할 수 있었습니다. 더 나아가 연령대 별로 얼마나 높은 치명률을 가지고 있는지 확진자 수 대비 사망률을 구하여 연령별의 정확한 치명률을 구하였습니다. 그리고 시간의 흐름에 따른 연령별 치명률을 알아보겠습니다.

치명률  = 사망자수 / 확진자 수 *100

<b> 단, 0대부터 20대(0살~29살) 까지 연령에는 사망자가 있지 않아 치명률 분석에서 제외하였습니다. </b>

In [30]:
timeAge

,date,time,age,confirmed,deceased
0,2020-03-02,0,0s,32,0
1,2020-03-02,0,10s,169,0
2,2020-03-02,0,20s,1235,0
3,2020-03-02,0,30s,506,1
4,2020-03-02,0,40s,633,1
...,...,...,...,...,...
535,2020-04-30,0,40s,1427,3
536,2020-04-30,0,50s,1956,15
537,2020-04-30,0,60s,1348,35
538,2020-04-30,0,70s,709,74


In [31]:
timeAge['deathrate'] = timeAge['deceased'] / timeAge['confirmed'] * 100

In [32]:
mask = timeAge['age'].isin(['0s','10s','20s'])
timeAge = timeAge[~mask]

In [33]:
layout_setting = {'xaxis_title':'<b>확진자 수</b>','yaxis_title':'<b>사망자 수(log)</b>',
                  'font':dict(size=18,color='#60606e',family='Franklin Gothic' )}
fig = px.scatter(timeAge, x="confirmed", y="deceased", 
                 animation_frame="date", animation_group="age",
                 size="deathrate", color="age", hover_name="age",
                 log_x=False, size_max=100, range_x=[30,2000],
                range_y=[0,3])

fig["layout"].pop("updatemenus") 
fig.update_layout(title='<b>시간에 따른 연령별 치명률 변화<b>', **layout_setting, 
                   yaxis_type="log",
                  xaxis=dict(showgrid=False, zeroline=False),
                  yaxis=dict(showgrid=False, zeroline=False))
fig.show()

질병관리본부에 따르면 2020년 02월 09일 첫 사망 사례 이후 4월 30일 0시 까지 총 247명의 사망자가 발생하였고, 우리나라의 치명률은 전국 확진자  (10,765명 대비) 2.29%입니다. 

기간에 따른 연령별 치명률을 보고자 2020/03/02부터 2020/04/30까지 기간의 timeAge.csv를 사용하였습니다. 

우선, 2020/03/02 부터 슬라이드를 보게 되면 70대와 80대의 치명률이 각각 3.7%, 3.1% 로 높지 않을 뿐만 아니라 고연령층 내의 차이가 크게 나지 않습니다.
하지만 시간이 흐를수록 점차 80대의 치명률이 급격히 높아지며 2020/04/30 기준으로 80대 이상의 치명률은 24.3% 로써 4명중 1명은 사망했다고 할 수 있을 정도로 매우 높은 비율을 보여줍니다. 그 외 70대 10.4%, 60대 2.6%, 50대 0.8%, 40대 0.21%, 30대 0.17% 의 치명률의 차이를 볼 수 있습니다.

### 2-2) 사망자 원인 분석 

PatientInfo.csv내 사망 환자 정보 60건의 질병의 유무가 사망에 큰 영향을 끼쳤는지 알아보았으나 PatientInfo내부의 질병 유무는 대부분 NAN값으로 되어있었습니다. 이를 뒷받침하기 위해 질병관리본부의 4월 30일자 브리핑 데이터를 가져와 질병유무를 정확하게 판단하였습니다.

우선, PatientInfo 데이터에서 state가 deceased인 환자의 정보를 뽑아보니 총 60명의 사망자 정보가 나왔습니다. 존재하는 사망 환자 정보를 통해 사망 환자의 공통점과 차이점을 통해 원인을 찾아 보고자 하였습니다.

#### 기저질환을 앓고 있었던 사망자들

1.  사망 환자를  탐색하던 중 사망 기간이 -1 인 데이터가 2건 있었습니다.

    -1인 사망 환자를 뽑아 본 결과 사후판정 받은 청도 대남 병원 환자임을 확인할 수 있었습니다.
    2건 모두 질병여부가 있었으며, 청도 대남 병원에서 각  2월 19일, 2월 21일에 사망하였습니다.

    - 첫번째 사후 판정 사망자 : 청도 대남 정신병동에서 20년 넘게 질병을 치료 중이었던 페렴 환자(60대 남성) 였으며,  사망 후 코로나 사후확진 판정을 받았음을 확인하였습니다.

    - 두번째 사후 판정 사망자 :  50대 여성 역시 페렴을 앓고 있었던 여성이며 , 대남 병원에 오래 입원해 있던 환자였습니다.  부산대 병원으로 이송되었지만 도착 직후 숨졌습니다.


2.  두 명 사망자 외 청도 대남 병원에 총 7명의 사망환자가 있었으며, 모두 다음과 같은 공통점이 있었습니다.

    - 50,60대의 장년층이라는 공통점.

    - 본래 질병을 가진 중증환자라는 공통점. 

    - 확진후 평균 1.8일만에 사망했다는 공통점.
    
    
3. 대구지역 20건, 경상북도 37건, 강원도 1건, 경기도 1건, 울산 1건 총 60명의 사망자의 공통점은 다음과 같습니다.

    - 사망 환자들은 확진 판정후 평균 8일만에 사망하였으며 질병이 있는 환자들은 평균 2.5일만에 사망합니다.
  
    - 사망 환자 중 기저질환을 앓고 있었던 환자가 약 30%정도 있다는 것을 확인했을 때 기저질환을 앓고 있는 사람들이 사망률이 높다라는 결론을 내세우긴 조금 애매하였습니다. 그래서 질병관리본부 데이터를 활용해 사망자의 질병유무에 대한 근거를 뒷받침 하였습니다.


In [34]:
patientinfo[patientinfo['state']=='deceased']

,patient_id,global_num,sex,birth_year,age,country,province,city,disease,infection_case,infection_order,infected_by,contact_number,symptom_onset_date,confirmed_date,released_date,deceased_date,state
108,1000000109,6773.0,male,1929.0,90s,Korea,Seoul,etc,NaN,contact with patient,NaN,NaN,NaN,2020-03-02,2020-03-07,NaN,NaN,deceased
284,1000000285,8602.0,male,1976.0,40s,Korea,Seoul,Mapo-gu,NaN,Guro-gu Call Center,NaN,NaN,NaN,NaN,2020-03-19,NaN,NaN,deceased
703,1100000071,NaN,male,1941.0,70s,Korea,Busan,Busanjin-gu,NaN,etc,NaN,NaN,1.0,2020-02-28,2020-02-28,NaN,NaN,deceased
727,1100000095,NaN,female,1932.0,80s,Korea,Busan,etc,NaN,NaN,NaN,NaN,24.0,2020-02-15,2020-03-13,NaN,NaN,deceased
729,1100000097,NaN,male,1947.0,70s,Korea,Busan,Busanjin-gu,NaN,etc,NaN,NaN,11.0,2020-03-09,2020-03-13,NaN,NaN,deceased
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3151,6020100107,107.0,male,1953.0,60s,Korea,Gyeongsangbuk-do,Cheongdo-gun,True,Cheongdo Daenam Hospital,NaN,NaN,NaN,NaN,2020-02-21,NaN,2020-02-24,deceased
3152,6020100205,205.0,female,1965.0,50s,Korea,Gyeongsangbuk-do,Cheongdo-gun,True,Cheongdo Daenam Hospital,NaN,NaN,NaN,NaN,2020-02-22,NaN,2020-02-21,deceased
3153,6020100286,286.0,male,1958.0,60s,Korea,Gyeongsangbuk-do,Cheongdo-gun,True,Cheongdo Daenam Hospital,NaN,NaN,NaN,NaN,2020-02-22,NaN,2020-02-23,deceased
3154,6020100298,298.0,male,1962.0,50s,Korea,Gyeongsangbuk-do,Cheongdo-gun,True,Cheongdo Daenam Hospital,NaN,NaN,NaN,NaN,2020-02-22,NaN,2020-02-25,deceased


In [35]:
deceased_p=patientinfo[patientinfo['state']=='deceased']
deceased_p=deceased_p.dropna(subset=['deceased_date'], inplace=False)

deceased_p['confirmed_date']=pd.to_datetime(deceased_p['confirmed_date'])
deceased_p['deceased_date']=pd.to_datetime(deceased_p['deceased_date'])
deceased_p['period_diff']=(deceased_p['deceased_date'])-(deceased_p['confirmed_date'])

deceased_p['period_diff'].describe()

count                         62
mean      8 days 17:01:56.129032
std      10 days 16:40:08.908869
min            -1 days +00:00:00
25%              2 days 00:00:00
50%              5 days 00:00:00
75%             11 days 00:00:00
max             48 days 00:00:00
Name: period_diff, dtype: object

In [36]:
deceased_60p=deceased_p.loc[:,['patient_id','sex','age','province','city','disease','deceased_date','period_diff']] 

In [37]:
deceased_60p['period_diff']=deceased_60p['period_diff'].astype(str)
deceased_60p['deceased_date']=deceased_60p['deceased_date'].astype(str)

In [38]:
import re

diff_date_list=[]
province_list=[]
city_list=[]

for diff in deceased_60p['period_diff']:
    pattern=re.compile('[-]*[\d*]{1,2}')
    word=re.search(pattern=pattern,string=diff)
    diff_date_list.append(word.group())
    
for p in deceased_60p['province']:
    pattern=re.compile('[-]')
    word=re.sub(pattern=pattern,repl='',string=p)
    province_list.append(word)
     
deceased_60p['period_diff']=diff_date_list
deceased_60p['province']=province_list
deceased_60p['province']=np.where(deceased_60p['province']=='Gyeongsangbukdo','Gyeongbuk',deceased_60p['province'])

In [39]:
table = ff.create_table(deceased_60p[deceased_60p['disease']==True])
plotly.offline.iplot(table) 

60명외 사망자에 관해 더 자세하게 알아보기 위해 질병관리본부 4월 30일자 브리핑 데이터를 가져와 사망 원인을 파악하였습니다.

#### 기저질환을 가지고 있었던 사망자 및 추정 감염경로

첫번째 시각화는 사망자들이 가지고 있었던 기저질환의 종류를 알아보았습니다. 
기저질환을 가지고 있었던 사망자는 98.8%(약 99%)으로 압도적인 수치를 통해 다시 한번 기저질환을 가진 확진자는 사망률이 매우 높다는 사실을 인지하였습니다. 하지만, 기저질환을 가지지 않은 환자가 사망한 사례도 1건 존재하였습니다. 이를 통해 기저질환을 가지고 있지 않다고 하여도 사망할 수 있는 가능성은 존재함으로써 위험성을 잊어서는 안됩니다.
 또한 사망자들이 모두 폐와 관련 기저질환을 앓고 있어 사망했을 것이다라고 생각하였으나 호흡 기계 질환보다 순한기계 질환을 앓고 있었던 사망자들이 76.9%로 3배이상 차이를 보이며 사망률과 더 밀접한 관련이 있음을 확인하였습니다. 다음은 시각화의 이해를 돕기 위해 해당 기저 질환의 세부 병명을 기재하였습니다.


- 순한기계 질환 : 심근경색, 심부전, 뇌졸중, 고혈압 등
- 내분비계 질환 : 당뇨병, 통풍, 쿠싱증후군 등
- 정신 질환 : 치매, 조현병 등
- 호흡 기계 질환 : 만성 패쇄성 폐 질환, 천식 등
- 비뇨.생식 기계 질환 : 만성신장질환, 전립선질환 등
- 악성신생물(암) : 폐암, 간암, 위암 등
- 신경계 질환 : 피킨슨 병 등
- 소화기계 질환 : 간경변증 등
- 근골격게 질환 : 골다골증, 관절암 등
- 혈액 및 조혈계 질환 : 원발성 혈소판증가증, 빈혈 등

두번째 시각화는 사망자의 감염 경로를 나타내였습니다. 50%이상이 의료시설에서의 감염이 일어난 것을 보아 기존에 기저질환을 앓고 있던 환자가 병원을 이용하던 도중 확진자와 접촉하여 감염이 되었을 가능성이 큽니다. 이에 따라 기존 기저질환을 가진 환자들을 위해 확진자가 있는 병원은 이용을 막고, 기저질환을 가진 환자들과 기존 코로나 확진자들의 접촉을 최대한 막아야 사망률을 낮출 수 있을거라 생각합니다.

In [40]:
disease = pd.DataFrame({'기저질환 종류' : ['순환기계 질환','내분비계 대사성 질환','정신 질환','호흡기계 질환', '비뇨 생식기계 질환','악성신생물(암)','신경계 질환', '소화기계 질환','근골격계 질환','혈액 및 조혈계 질환'],
             'Count': [190, 122, 109, 62, 44, 33, 16, 11, 6, 5],
             'Per': ['76.9%', '49.4%', '44.1%','25.1%','17.8%','13.4%','6.5%','4.5%','2.4%','2.0%']})

In [41]:
color = ['#FFFFFF', px.colors.sequential.Reds[3], px.colors.sequential.Reds[1], px.colors.sequential.Reds[1],
         px.colors.sequential.Reds[1], px.colors.sequential.Reds[1], px.colors.sequential.Reds[1],
         px.colors.sequential.Reds[5], px.colors.sequential.Reds[3], px.colors.sequential.Reds[8]]

fig = make_subplots(1, 2, specs=[[{"type": "xy"}, {"type": "domain"}]], subplot_titles=['<b>기저질환 종류별 사망자(단위: 명/중복 포함)</b>', '<b>사망자의 추정 감염 경로(단위: 명)</b>'], 
                      vertical_spacing=0.5)

fig.add_trace(go.Bar(x=disease['기저질환 종류'],y=disease['Count'], 
                     marker=dict(color=px.colors.sequential.Emrld[4]),
                    ),1,1)

fig.add_trace(go.Sunburst(
    labels=["전체 사망자 수", "시설 및 병원", "청도 대남 병원", "요양 병원", 
               "기타 의료기관", "요양원", "기타 사회복지 시설", "신천지 관련", "확진자 접촉", "조사중"],
    
    parents=["", "전체 사망자 수", "시설 및 병원", "시설 및 병원", "시설 및 병원", "시설 및 병원", "시설 및 병원", 
            "전체 사망자 수", "전체 사망자 수","전체 사망자 수"],
    values=[247, 132, 9, 64, 30, 20, 9, 26, 21, 68],
    marker=dict(colors= color),
    textinfo='label+value+percent entry',
    branchvalues='total',
   ),1,2) 

fig.update_layout(margin = dict(l=0, r=0, t=30, b=0), height = 500)
fig.show()

# 최종 결론

사망률을 줄이기 위해서는 <span style="color:red">70대이상의 완치기간이 긴 노년층</span>의 주의있는 예방이 필요합니다. 또한 페렴 뿐만아니라 <span style="color:red"> 기저질환이 있는 환자들</span>에게 매우 치명적임으로 병원에 대한 항시 방역과 기존 감염자와의 접촉을 최대한 막아야 코로나 바이러스로 인한 사망률을 줄일 수 있을 것입니다.